#使い方
上から順番にコードセルを実行していくだけです。（それぞれ処理終了まで待つこと）

各コードセル上部の説明を読んでから実行して下さい。

In [ ]:
#####環境構築#####
# 環境構築完了と表示されるまでお待ち下さい。

%tensorflow_version 1.x
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/NVIDIA/tacotron2.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  !cd {project_name}/waveglow && git checkout 9168aea
  !pip install -q librosa unidecode

import sys
sys.path.append(join(project_name, 'waveglow/'))
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

!gdown https://drive.google.com/uc?id=1fMavZ6ZO0_D7MdfvZD1cQMD9H7GwCMNo
!gdown https://drive.google.com/uc?id=1WsibBTsuRg_SF2Z6L6NFRTT-NjEy1oTx

print("----------環境構築完了----------")

In [ ]:
#####学習済モデルのロード#####
# ロード完了と表示されるまでお待ち下さい。

import IPython.display as ipd
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT
from audio_processing import griffin_lim
from text import text_to_sequence
from denoiser import Denoiser

torch.set_grad_enabled(False)

hparams = create_hparams()
hparams.sampling_rate = 22050
model = Tacotron2(hparams)
model.load_state_dict(torch.load('/content/ver_1_0_0')['state_dict'])
_ = model.cuda().eval()

import json
from glow import WaveGlow
waveglow_config = json.load(open('%s/waveglow/config.json' % project_name))['waveglow_config']
waveglow = WaveGlow(**waveglow_config)
waveglow.load_state_dict(torch.load('/content/waveglow_256channels_ljs_v2.pt')['model'].state_dict())
_ = waveglow.cuda().eval()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

print("----------ロード完了----------")

In [ ]:
#####音声の生成#####
# 右のテキストボックスに生成したい音声のテキストを入力して実行して下さい。
# ヘボン式ローマ字及びコンマ、ピリオドのみなので注意！

#@title テキスト（ヘボン式ローマ字で入力！）
TEXT = "namamuginamagomenamatamago." #@param {type:"string"}
sequence = np.array(text_to_sequence(TEXT, ['basic_cleaners']))[None, :]
sequence = torch.autograd.Variable(torch.from_numpy(sequence)).long()
sequence = sequence.cuda()
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)